In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
from random import sample
import random
drive.mount('/content/drive', force_remount=True)

slide_num = 100 #連續期數
repeat_time = 30 #重複抽取次數
print("Start")
print("Slide number: ", slide_num)
print("Repeat number: ", repeat_time)

# put your data path, ex: '/content/drive/MyDrive/.../'
input_data_machine = '/.../data_1212.xlsx'
input_data_rate = '/.../1212瞬測儀資料.xlsx'
data_machine = pd.read_excel(input_data_machine, usecols=["package","frequency","Speed","Status"])
data_rate = pd.read_excel(input_data_rate,'不良率＿用這個表', usecols=["編碼","總不良率"])
data_machine = data_machine.dropna()
data_rate = data_rate.dropna()

print(data_machine.head())
print(data_rate.head())
print(data_machine.shape)
print(data_rate.shape)

Mounted at /content/drive
Start
Slide number:  100
Repeat number:  30
   Speed  Status  frequency                     package
0  361.0     2.0   3.000000  20200809_101180441_D-008_1
1  361.0     2.0   3.500000  20200809_101180441_D-008_1
2  361.0     2.0   3.516667  20200809_101180441_D-008_1
3  250.0     4.0   1.800000  20200809_101180441_D-008_1
4  250.0     2.0   0.050000  20200809_101180441_D-008_1
                           編碼  總不良率
0  20200810_101180441_D-008_0  0.00
1  20200810_101180441_D-008_0  0.00
2  20200810_101180441_D-008_0  0.00
3  20200810_101180441_D-008_0  0.00
4  20200810_101180441_D-008_1  0.25
(351091, 4)
(7476, 2)


In [2]:
"""
將瞬測儀和機台數據資料分包&配對
"""
data_machine.set_index("package", inplace=True)
all_data_machineId = np.array(data_machine.index.drop_duplicates(keep='first').values)
data_machine.reset_index(inplace=True)

data_rate.set_index("編碼", inplace=True)
all_data_rateId = np.array(data_rate.index.drop_duplicates(keep='first').values)
data_rate.reset_index(inplace=True)

print(all_data_machineId.size)
print(all_data_rateId.size)

pkg_num = 0

#編號一致
for data_machineId in all_data_machineId:
  for data_rateId in all_data_rateId:
    if data_machineId == data_rateId:
      pkg_num += 1 
      globals()['x_'+str(pkg_num)] = data_machine[data_machine["package"] == data_machineId]
      globals()['y_'+str(pkg_num)] = data_rate[data_rate["編碼"] == data_rateId]
    else :
      pass

max_pkg_num = pkg_num
print("Total package number: ", max_pkg_num)

"""
計算筆數小於slide_num筆包數
"""
usable_pkg = 0
lessthan = 0

for pkg_num in range(1, max_pkg_num+1):
  if len(globals()['x_'+str(pkg_num)]) < slide_num:
    lessthan += 1
  else:
    if usable_pkg == 0:
      first_pkg = pkg_num 
    usable_pkg += 1
  globals()['x_'+str(pkg_num)] = globals()['x_'+str(pkg_num)].drop(["package"], axis=1)
  globals()['y_'+str(pkg_num)] = globals()['y_'+str(pkg_num)].drop(columns = ["編碼"])

print("Less than slide_num data package: ",lessthan)
print("Usable package: ",usable_pkg)
print("First package number: ",first_pkg)

1296
1233
Total package number:  1122
Less than slide_num data package:  145
Usable package:  977
First package number:  2


In [3]:
"""
處理feature & 各包抽slide_num*n筆, 處理label
"""
for pkg_num in range(1, max_pkg_num+1):
  pkg_size = len(globals()['x_'+str(pkg_num)])

  if len(globals()['x_'+str(pkg_num)]) < slide_num:
    pass
  else: 
    """
    slide repeat_time次, 挑連續slide_num筆feature data, shape = (包*repeat_time*slide_num ,3)
    """
    for num in range(repeat_time):
      end = pkg_size - slide_num
      random_start = random.randint(0,end)

      if num == 0:
        globals()['x_slide_'+str(pkg_num)] = globals()['x_'+str(pkg_num)][random_start:random_start+slide_num]
      else:
        globals()['x_slide_'+str(pkg_num)] = pd.concat([globals()['x_slide_'+str(pkg_num)], globals()['x_'+str(pkg_num)][random_start:random_start+slide_num]])

    """
    處理label, shape = (包*repeat_time, 1)
    """
    pkg_rate = globals()['y_'+str(pkg_num)].iloc[0].round(2).values
    slide_time = int(len(globals()['x_slide_'+str(pkg_num)])/slide_num)
    globals()['y_slide_'+str(pkg_num)] = np.zeros((slide_time,1))

    for y_num in range(slide_time):
      globals()['y_slide_'+str(pkg_num)][y_num] = np.array([pkg_rate])

print(globals()['x_slide_'+str(first_pkg)].shape)  
print(globals()['y_slide_'+str(first_pkg)].shape)

(3000, 3)
(30, 1)


In [4]:
"""
依序存進data & label
"""
for pkg_num in range(1, max_pkg_num+1):
  if len(globals()['x_'+str(pkg_num)]) < slide_num:
    pass
  else:
    if pkg_num == first_pkg:
      data = pd.DataFrame(globals()['x_slide_'+str(pkg_num)])    
      label = pd.DataFrame(globals()['y_slide_'+str(pkg_num)])    
    else:
      data = pd.concat([data, pd.DataFrame(globals()['x_slide_'+str(pkg_num)])])
      label = pd.concat([label, pd.DataFrame(globals()['y_slide_'+str(pkg_num)])])

data = data.values
label = label.values

print(data.shape) 
print(label.shape) 

(2931000, 3)
(29310, 1)


In [6]:
slide_size = 0
pkg_size = 0
data_temp = []

for pkg_num in range(1, max_pkg_num+1):
  if len(globals()['x_'+str(pkg_num)]) < slide_num:
    pass
  else: 
    for num in range(repeat_time): 
      data_temp.append(data[slide_size:(slide_size+slide_num), 0:3])
      slide_size += slide_num

data = np.array(data_temp)

print(data.shape)
print(label.shape)

(29310, 100, 3)
(29310, 1)


In [7]:
permutation=np.random.permutation(label.shape[0])
shuffled_data=data[permutation,:,:]
shuffled_label=label[permutation]

rate=0.7
X_train=shuffled_data[:int(shuffled_data.shape[0]*rate)]
Y_train=shuffled_label[:int(shuffled_label.shape[0]*rate)]
X_test=shuffled_data[int(shuffled_data.shape[0]*rate):]
Y_test=shuffled_label[int(shuffled_label.shape[0]*rate):]

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(20517, 100, 3)
(20517, 1)
(8793, 100, 3)
(8793, 1)
